In [2]:
from google.colab import drive,files
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install texthero

     |████████████████████████████████| 1.5 MB 8.0 MB/s 
     |████████████████████████████████| 241 kB 35.0 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [4]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import hashlib
import nltk

import texthero as hero
from texthero import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
input_path = '/content/drive/MyDrive/機械学習/Competitions/Signate/医学論文の自動仕分けチャレンジ/input'

In [6]:
all_df = pd.read_feather(os.path.join(input_path,'all_clean_df.feather'))
all_df.head()

,id,title,abstract,judgement,clean_title,clean_abstract
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0.0,one year age changes mri brain volumes older...,longitudinal studies indicate declines cogni...
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0.0,supportive csf biomarker evidence enhance na...,present study undertaken validate measurem...
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0.0,occurrence basal ganglia germ cell tumors wit...,objective report case series basal ganglia...
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0.0,new developments diagnosis therapy crohn d...,etiology pathogenesis idiopathic chronic in...
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,None,0.0,prolonged shedding sars cov elderly liver t...,


In [7]:
start = time.time()

sentences = []
print("Parsing sentences from traning set...")

#Loop over each news article
for review in tqdm(all_df['clean_abstract']):
  try:
    #Split a review into parsed sentences
    result = review
    h = result.split(" ")
    h = list(filter(("").__ne__,h))
    sentences.append(h)
  except:
    continue

num_features = 200
min_word_count = 20
num_workers = 1
context = 10
downsampling = 1e-3
seed = 42

hashfxn = lambda x: int(hashlib.md5(str(x).encode()).hexdigest(),16)
print("Training FastText model...")
#Train FastText model.
model = FastText(sentences,workers=num_workers,hs=0,sg=1,negative=10,iter=5,size=num_features,min_count=min_word_count,hashfxn=hashfxn,window=context,sample=downsampling,seed=seed)
model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" + str(context) + "context_len2alldata"

model.init_sims(replace=True)

endmodeltime = time.time()

print("time: ",endmodeltime-start)

Parsing sentences from traning set...


100%|██████████| 67979/67979 [00:02<00:00, 28170.08it/s]


Training FastText model...
time:  2010.0570659637451


In [8]:
tqdm.pandas()

def des_to_mean_vec(text):

  try:
    text = text.strip()
    text_ls = [s for s in text.split(' ') if '' != s]
    return  np.mean([model.wv[word] for word in text_ls if word in model.wv.index2word],axis=0)
  except:
    return np.nan

df = all_df['clean_abstract'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

  0%|          | 0/67979 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 67979/67979 [09:00<00:00, 125.73it/s]


0    [0.017403029, -0.014961936, -0.029320827, 0.03...
1    [-0.013803987, 0.007566454, -0.023452222, 0.01...
2    [0.015309222, -0.033253822, -0.0251176, 0.0335...
3    [0.018104736, -0.049468733, -0.05605503, 0.034...
4                                                  NaN
Name: clean_abstract, dtype: object

In [9]:
abstract_df = df.apply(pd.Series)
abstract_df.columns = ['fasttext_abstract_' + str(col) for col in abstract_df.columns]
abstract_df.head()

,fasttext_abstract_0,fasttext_abstract_1,fasttext_abstract_2,fasttext_abstract_3,fasttext_abstract_4,fasttext_abstract_5,fasttext_abstract_6,fasttext_abstract_7,fasttext_abstract_8,fasttext_abstract_9,fasttext_abstract_10,fasttext_abstract_11,fasttext_abstract_12,fasttext_abstract_13,fasttext_abstract_14,fasttext_abstract_15,fasttext_abstract_16,fasttext_abstract_17,fasttext_abstract_18,fasttext_abstract_19,fasttext_abstract_20,fasttext_abstract_21,fasttext_abstract_22,fasttext_abstract_23,fasttext_abstract_24,fasttext_abstract_25,fasttext_abstract_26,fasttext_abstract_27,fasttext_abstract_28,fasttext_abstract_29,fasttext_abstract_30,fasttext_abstract_31,fasttext_abstract_32,fasttext_abstract_33,fasttext_abstract_34,fasttext_abstract_35,fasttext_abstract_36,fasttext_abstract_37,fasttext_abstract_38,fasttext_abstract_39,...,fasttext_abstract_160,fasttext_abstract_161,fasttext_abstract_162,fasttext_abstract_163,fasttext_abstract_164,fasttext_abstract_165,fasttext_abstract_166,fasttext_abstract_167,fasttext_abstract_168,fasttext_abstract_169,fasttext_abstract_170,fasttext_abstract_171,fasttext_abstract_172,fasttext_abstract_173,fasttext_abstract_174,fasttext_abstract_175,fasttext_abstract_176,fasttext_abstract_177,fasttext_abstract_178,fasttext_abstract_179,fasttext_abstract_180,fasttext_abstract_181,fasttext_abstract_182,fasttext_abstract_183,fasttext_abstract_184,fasttext_abstract_185,fasttext_abstract_186,fasttext_abstract_187,fasttext_abstract_188,fasttext_abstract_189,fasttext_abstract_190,fasttext_abstract_191,fasttext_abstract_192,fasttext_abstract_193,fasttext_abstract_194,fasttext_abstract_195,fasttext_abstract_196,fasttext_abstract_197,fasttext_abstract_198,fasttext_abstract_199
0,0.017403,-0.014962,-0.029321,0.038656,-0.039083,-0.013778,-0.014483,-0.006071,-0.020733,0.004315,-0.005177,-0.036574,-0.004974,-0.032778,-0.012672,0.010655,-0.084869,0.036910,0.031685,-0.031293,-0.039522,0.040059,-0.031437,-0.012389,-0.023374,-0.013059,-0.000930,-0.012031,-0.017024,0.012648,0.054290,0.012514,-0.027036,-0.070310,-0.042366,0.013865,-0.034058,-0.017177,0.012634,0.016227,...,0.044946,0.039480,-0.073520,0.060971,0.051855,-0.024510,0.053443,-0.093302,0.015325,-0.005101,0.080898,0.098735,0.017490,-0.007093,0.057646,0.042636,-0.038823,0.024608,-0.064787,-0.026663,0.014521,-0.003810,0.055582,-0.037770,0.067299,-0.006145,0.009564,-0.043258,0.039097,0.022735,0.053088,-0.066442,0.066734,-0.007594,-0.021980,0.018253,0.025317,-0.062365,-0.056317,0.018135
1,-0.013804,0.007566,-0.023452,0.017058,-0.007369,-0.009546,-0.010447,-0.001565,-0.035673,0.017357,-0.002719,-0.042666,-0.022459,-0.055874,-0.018680,-0.001000,-0.096095,0.063582,0.012535,-0.042953,-0.041830,0.045205,-0.021487,0.017233,-0.014213,-0.032064,0.003144,-0.045371,-0.020445,0.044426,0.029811,0.006962,-0.007342,-0.069758,-0.053226,0.020217,-0.049190,-0.041496,0.054316,-0.022422,...,0.028044,0.042307,-0.029404,0.049550,0.022622,-0.029323,0.057755,-0.073232,0.045522,-0.014523,0.050826,0.069454,0.041479,0.042423,0.041252,0.066645,-0.016851,0.034333,-0.021329,-0.011002,0.022033,-0.005356,0.039473,-0.053713,0.037108,0.012443,0.026020,-0.021501,0.032966,0.002299,0.031399,-0.078896,0.088841,0.006586,-0.035081,0.012587,0.024500,-0.009744,-0.027444,0.045864
2,0.015309,-0.033254,-0.025118,0.033531,-0.020007,0.010025,-0.017142,-0.018444,0.011565,-0.003819,-0.013322,-0.018240,0.021653,-0.052051,0.007001,0.035747,-0.057355,0.044615,0.001197,-0.029551,-0.024519,0.034513,-0.038738,0.000950,-0.022956,-0.013458,-0.022274,-0.026398,-0.021492,0.009437,0.028671,0.000496,-0.015014,-0.082518,-0.060144,-0.017959,-0.008925,-0.026155,0.067336,0.006880,...,0.022678,0.027433,-0.045743,0.024422,0.074522,-0.044440,0.037339,-0.063240,0.017570,-0.028712,0.112160,0.080814,0.005896,0.006256,0.061930,0.028394,0.007982,0.029760,-0.050083,-0.030703,0.023211,0.003079,0.051424,-0.021758,0.036788,-0.001405,0.009488,-0.059650,0.038512,-0.009385,0.019141,-0.069179,0.051743,0.010269,-0.044265,0.014736,0.013261,-0.032241,-0.

In [10]:
abstract_df.to_feather(os.path.join(input_path,'abstract_fasttxt_df.feather'))

In [11]:
file = os.path.join(input_path,'fasttext_model_abstract.dill')
dill.dump(model,open(file,'wb'))